### Dependancies used

In [1]:
import sys

sys.path.append("/Users/louie/Documents/Visual Studio Code/ACT Work/Airbus-Aircraft-Detection/py")

from functions import *

Hello, World!


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import ast

from PIL import Image,ImageDraw

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models import resnet50, ResNet50_Weights


from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#HOME = "/Users/louie/Documents/Visual Studio Code/ACT Work/Airbus-Aircraft-Detection"
HOME = '/content/drive/MyDrive/ACT Coursework 2/Airbus-Aircraft-Detection/'
directory_path = os.path.join(HOME, 'airbus-aircrafts-sample-dataset')
image_path = os.path.join(directory_path, 'images')

### What is data augmentation?

Data augmentation is the process of generating more data out of a pre-existing dataset using various transformations to create new data, which can increase dataset diversity for training a machine learning algorithm.

### Why use data augmentation?

Data augmentation is very useful in situations where gathering more real data has ethical, financial or time restrictions and the current dataset is insufficient for accurate training and the diversity created by the adds more general features that help the model expect differences in features and better prepares it for unseen data which improves its overall performance.

In the case of the AirBus dataset being used, the dataset is quite small and both the desired and undesired features are very repetitive which can lead to overfitting as shown by the loss curve in out neural network.

### Methods of data augmentation

Data augmentation can be performed in a variety of ways to improve model diversity including but not limited to...
- Rotation: rotating images can help the model recognise models at different angles
- Cropping: removing sections of the image can simulate a closer image or focus on a particular feature
- Brightness: changing the brightness of the image can simulate variations in lighting conditions
- Gaussian Noise: adding noise can blur the image and simulate sensor noise and graininess.

Lets demonstrate this by taking a random data sample and using Albumentations to apply a [Gaussian Blur](https://en.wikipedia.org/wiki/Gaussian_blur) transformation to it.